In [ ]:
# DataSens E1_v1 - 03_ingest_sources
from faker import Faker
import pandas as pd
from pathlib import Path

fake = Faker()
rows = [
    {"user_id": i, "email": fake.email()} for i in range(1, 101)
]
df = pd.DataFrame(rows)
Path("data/raw").mkdir(parents=True, exist_ok=True)
df.to_csv("data/raw/users.csv", index=False)
print("Simulated data written to data/raw.")
